<a href="https://colab.research.google.com/github/mary-ts/Neural-Computing/blob/main/xceptionGoodFruity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.layers as layers
from keras import Sequential
import keras.models
from keras.layers import *
from keras.models import * 
from keras import optimizers, applications
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping

Dataset Paths

In [4]:
train_path = "/content/drive/MyDrive/CNN IMAGES/archive (1)/val/train"
test_path = "/content/drive/MyDrive/CNN IMAGES/archive (1)/val/val"

Creating training and test data

In [5]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
test_datagen= image.ImageDataGenerator(    
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [6]:
!pip install split-folders
import splitfolders

# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/content/drive/MyDrive/CNN IMAGES/archive (1)/train/train", output="/content/drive/MyDrive/CNN IMAGES/archive (1)/val",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Copying files: 16854 files [05:50, 48.09 files/s] 


Importing data into generators

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (299,299),
    batch_size = 8,
    class_mode = 'categorical', 
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (299,299),
    batch_size = 8,
    shuffle=True,
    class_mode = 'categorical')

Found 13471 images belonging to 33 classes.
Found 3383 images belonging to 33 classes.


Xception Model

In [8]:
model = applications.Xception(weights='imagenet', 
                              include_top=False, 
                              input_shape=(299, 299, 3))

for layer in model.layers[:-5]:
    layer.trainable = False

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(.3)(x)
predictions = Dense(33, activation="softmax")(x)
model_final = Model(model.input, predictions)

model_final.compile(optimizers.RMSprop(learning_rate=0.0001, decay=1e-5),loss='categorical_crossentropy',metrics=['accuracy'])
#model_final.summary()

83683744/83683744 [==============================] - 1s 0us/step


Xception


In [ ]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=7, verbose=2, mode='auto')
hist = model_final.fit_generator(generator=train_generator,                   
                                    steps_per_epoch=150,
                                    validation_data=test_generator,                    
                                    validation_steps=10,
                                    epochs=25,
                                    callbacks = [checkpoint],
                                    verbose=2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/25

Epoch 1: val_loss improved from inf to 1.81729, saving model to vgg16_1.h5
150/150 - 56s - loss: 3.0566 - accuracy: 0.2625 - val_loss: 1.8173 - val_accuracy: 0.4500 - 56s/epoch - 376ms/step
Epoch 2/25

Epoch 2: val_loss improved from 1.81729 to 1.11587, saving model to vgg16_1.h5
150/150 - 39s - loss: 1.7564 - accuracy: 0.5100 - val_loss: 1.1159 - val_accuracy: 0.6500 - 39s/epoch - 260ms/step
Epoch 3/25

Epoch 3: val_loss improved from 1.11587 to 0.88812, saving model to vgg16_1.h5
150/150 - 40s - loss: 1.3223 - accuracy: 0.6155 - val_loss: 0.8881 - val_accuracy: 0.7875 - 40s/epoch - 266ms/step
Epoch 4/25

Epoch 4: val_loss improved from 0.88812 to 0.51309, saving model to vgg16_1.h5
150/150 - 39s - loss: 0.9952 - accuracy: 0.6967 - val_loss: 0.5131 - val_accuracy: 0.8250 - 39s/epoch - 260ms/step
Epoch 5/25

Epoch 5: val_loss did not improve from 0.51309
150/150 - 32s - loss: 0.9006 - accuracy: 0.7458 - val_loss: 0.6337 - val_accuracy: 0.7875 - 32s/epoch - 215ms/step
Epoch 

Plot Xception

In [ ]:
import matplotlib.pyplot as plt
#plt.plot(hist.history["loss"])
plt.plot(hist.history["accuracy"])
#plt.plot(hist.history["val_loss"])
plt.plot(hist.history["val_accuracy"])
plt.legend(["accuracy", "val_accuracy"])
plt.show()